# Train Models Using LeRobot on MI300x

This guide walks you through setting up environment for training imitation learning policies using LeRobot library on a DigitalOcean (DO) instance equipped with AMD MI300x GPUs and ROCm.

## ⚙️ Requirements

- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=${HF_USER}/${DATASET_NAME}`).
  If you don’t have an access token yet, you can sign up for Hugging Face [here](https://huggingface.co/join). After signing up, create an access token by visiting [here](https://huggingface.co/settings/tokens).
- A wandb account to enable training visualization and upload your training evidence to our github.
  You can sign up for Wandb [here](https://wandb.ai/signup) and visit [here](https://wandb.ai/authorize) to create a token.
- Access to DO instance AMD Mi300x GPU


## Verify ROCm and GPU availability

This cell uses `pytorch` to check AMD GPU Info. The expected ouput is

```
CUDA compatible device availability: True
device name [0]: AMD Instinct MI300X VF
```


In [ ]:
import torch

print(f"CUDA compatible device availability:", torch.cuda.is_available())
print(f"device name [0]:", torch.cuda.get_device_name(0))


CUDA compatible device availability: True
device name [0]: AMD Instinct MI300X VF


## Install FFmpeg 7.x

This cell uses `apt` to install ffmpeg 7.x for LeRobot.


In [51]:
!add-apt-repository ppa:ubuntuhandbook1/ffmpeg7 -y # install PPA which contains ffmpeg 7.x
!apt update && apt install ffmpeg -y

Repository: 'Types: deb
URIs: https://ppa.launchpadcontent.net/ubuntuhandbook1/ffmpeg7/ubuntu/
Suites: noble
Components: main
'
Description:
unofficial build for FFmpeg 7 for Ubuntu 22.04 | 24.04, backport from Debian's deb.multimedia.org repository

If the packages here are helpful, you may buy me a coffee:

         https://ko-fi.com/ubuntuhandbook1
More info: https://launchpad.net/~ubuntuhandbook1/+archive/ubuntu/ffmpeg7
Adding repository.
Found existing deb entry in /etc/apt/sources.list.d/ubuntuhandbook1-ubuntu-ffmpeg7-noble.sources
Hit:1 https://repo.radeon.com/amdgpu/30.10/ubuntu jammy InRelease
Hit:2 https://repo.radeon.com/rocm/apt/7.0 jammy InRelease                     
Hit:3 https://repo.radeon.com/graphics/7.0/ubuntu jammy InRelease              
Hit:4 http://archive.ubuntu.com/ubuntu noble InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]        
Get:6 http://security.ubuntu.com/ubuntu noble-security InRelease [126 

## Install LeRobot v0.4.1

This cell clones the `lerobot` repository from Hugging Face, and installs the package in editable mode. Extra Features: To install additional dependencies for training SmolVLA or Pi models, refer to the [LeRobot offical page](https://huggingface.co/docs/lerobot/index).


In [53]:
!git clone https://github.com/huggingface/lerobot.git
!cd lerobot && git checkout -b v0.4.1 v0.4.1 # let’s synchronize using this version
!cd lerobot && pip install -e .[smolvla]

fatal: destination path 'lerobot' already exists and is not an empty directory.
fatal: a branch named 'v0.4.1' already exists
Obtaining file:///workspace/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 84.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 109.9 MB/s  0:00:00
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.4.1-0.editable-py3-none-any.whl size=15631 sha256=e1a74674eac9e4260e4cde92c351ab3d22bf428a666293761a6a49ff59fc826d
  Stored in directory: /tmp/pip-ephem-wheel-cache-mv247b40/wheels/05/0a/0d/80a4c08845345c44fe1e5f70929884983b90d85f46a77f76

## Weights & Biases login

This cell install and log into Weights & Biases (wandb) to enable experiment tracking and logging.


In [ ]:
!pip install wandb
import wandb

wandb.login(key="<WANDB_KEY>")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Login into Hugging Face Hub


In [ ]:
from huggingface_hub import login

login(token="<HF_KEY>")

In [ ]:
from huggingface_hub import HfApi, login, list_repo_tree
from datasets import load_dataset

repo_id = "giacomoran/hackathon_amd_mission2_black_sort"

# Method 1: List episode files from the repo structure
print("📁 Episode files in dataset:")
api = HfApi()
for item in list_repo_tree(repo_id=repo_id, repo_type="dataset", recursive=True):
    if "episode" in item.path.lower() or item.path.endswith(".parquet"):
        print(f"  {item.path}")

# Method 2: Load dataset and check episode info
print("\n📊 Loading dataset metadata...")
dataset = load_dataset(repo_id, split="train")

# Check for episode column
if "episode_index" in dataset.column_names:
    episodes = sorted(set(dataset["episode_index"]))
    print(f"\n✅ Found {len(episodes)} episodes:")
    for ep in episodes:
        ep_data = dataset.filter(lambda x: x["episode_index"] == ep)
        print(f"  Episode {ep}: {len(ep_data)} frames")
else:
    print(f"\nDataset columns: {dataset.column_names}")
    print(f"Total rows: {len(dataset)}")

## Start Training Models with LeRobot

This cell uses the lerobot-train CLI from the lerobot library to train a robot control policy.

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `lerobot/svla_so100_pickplace`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.The name typically includes the model type (e.g., act, smolvla), the dataset name, and additional descriptive tags.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an AMD or NVIDIA GPU.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

7. `--policy.push_to_hub=`:

   Enables automatic uploading of the trained policy to the Hugging Face Hub. You must specify `--policy.repo_id` (e.g., ${HF_USER}/{REPO_NAME}) if it is True.


In [64]:
!rm -rf outputs/train/hackathon_amd_mission2_black_sort_smolvla

In [65]:
!lerobot-train \
  --dataset.repo_id=giacomoran/hackathon_amd_mission2_black_sort \
  --rename_map='{"observation.images.top": "observation.images.camera1", "observation.images.wrist": "observation.images.camera2"}' \
  --policy.empty_cameras=1 \
  --batch_size=64 \
  --steps=20000 \
  --output_dir=outputs/train/hackathon_amd_mission2_black_sort_smolvla \
  --job_name=hackathon_amd_mission2_black_sort_smolvla \
  --policy.repo_id=giacomoran/hackathon_amd_mission2_black_sort_smolvla \
  --policy.device=cuda \
  --policy.path=lerobot/smolvla_base \
  --policy.push_to_hub=true \
  --wandb.enable=true \
  --policy.chunk_size=30 \
  --policy.n_action_steps=30 \
  --save_freq=5000

INFO 2025-12-13 13:32:02 ot_train.py:163 {'batch_size': 64,
 'checkpoint_path': None,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'affine': {'kwargs': {'degrees': [-5.0,
                                                                            5.0],
                                                                'translate': [0.05,
                                                                              0.05]},
                                                     'type': 'RandomAffine',
                                                     'weight': 1.0},
                                          'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         '

In [70]:
!ls outputs/train/hackathon_amd_mission2_black_sort_smolvla/checkpoints/last/pretrained_model

config.json
model.safetensors
policy_postprocessor.json
policy_postprocessor_step_0_unnormalizer_processor.safetensors
policy_preprocessor.json
policy_preprocessor_step_5_normalizer_processor.safetensors
train_config.json


In [73]:
!huggingface-cli upload giacomoran/hackathon_amd_mission2_black_sort_smolvla \
  outputs/train/hackathon_amd_mission2_black_sort_smolvla/checkpoints/last/pretrained_model \
  .

⚠️  Warning: 'huggingface-cli upload' is deprecated. Use 'hf upload' instead.
Start hashing 7 files.
Finished hashing 7 files.
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

  ...zer_processor.safetensors: 100%|██████████████| 7.54kB / 7.54kB            


  ...zer_processor.safetensors: 100%|██████████████| 7.54kB / 7.54kB            

  ...zer_processor.safetensors: 100%|██████████████| 7.54kB / 7.54kB            


Processing Files (2 / 2)      :   0%|              | 15.1kB /  907MB,   ???B/s  

  ...zer_processor.safetensors: 100%|██████████████| 7.54kB / 7.54kB            


  ...zer_processor.safetensors: 100%|██████████████| 7.54kB / 7.54kB            



  ...d_model/model.safetensors:   1%|▏             | 9.13MB /  907MB            

  ...zer_processor.safetensors: 100%|██████████████| 7.54kB / 7.54kB            


  ...zer_processor.safetensors: 100%|███████████

**Notes**:

- If using a local dataset, add `--dataset.root=/path/to/dataset`.
- Adjust `--batch_size` and `--steps` based on your hardware and dataset.
- Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`
- Training progress visualized in your wandb dashboard


## Download Models from Hugging Face to Local Machine

Now after training is done, download the model to local machine.


In [32]:
!huggingface-cli download giacomoran/hackathon_amd_mission1 --repo-type model --local-dir outputs/train/hackathon_amd_mission1
# e.g. huggingface-cli upload ${HF_USER}/act_so101_3cube_1ksteps \
#  outputs/train/act_so101_3cube_1ksteps/checkpoints/last/pretrained_model

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 9 files:   0%|                                   | 0/9 [00:00<?, ?it/s]Downloading 'policy_preprocessor_step_3_normalizer_processor.safetensors' to 'outputs/train/hackathon_amd_mission1/.cache/huggingface/download/KTJX-ilfk9YZ9PyHew1aNVsbRpk=.75bebdcbfc0f385e5ac9989dbe6cc1d61b17283609bb5ad4a05911a6e6ad4727.incomplete'

policy_preprocessor_step_3_normalizer_pr(…):   0%|  | 0.00/7.54k [00:00<?, ?B/s]

config.json: 1.67kB [00:00, 12.3MB/s]A
Download complete. Moving file to outputs/train/hackathon_amd_mission1/config.json


policy_postprocessor.json: 100%|███████████████| 660/660 [00:00<00:00, 9.26MB/s]
Download complete. Moving file to outputs/train/hackathon_amd_mission1/policy_postprocessor.json


policy_preprocessor.json: 1.33kB [00:00, 3.77MB/s]A
Download complete. Moving file to outputs/train/hackathon_amd_mission1/policy_preprocessor.json


policy_postprocessor_step_0_unnormalizer(…):   0%|  

In [ ]:
!apt-get update && apt-get install -y zip

In [74]:
!tar -czvf wandb.tar.gz outputs/train/hackathon_amd_mission2_smallvla/wandb

tar: outputs/train/hackathon_amd_mission2_smallvla/wandb: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


## Miscs

1. Once the environment is setup, you can open a terminal session for training by navigating to `File → New Launcher → Other → Terminal`.
2. You can also upload your datasets to the container by clicking the `Upload Files` button in the left pane.


## Q&A

1. If you encounter an error like:
   ```
   FileExistsError: Output directory outputs/train/act_so101_3cube_1ksteps already exists and resume is False. Please change your output directory so that outputs/train/act_so101_3cube_1ksteps is not overwritten.
   ```
   Remove the existing directory before proceeding:


In [ ]:
!rm -fr outputs/train/hackathon_amd_mission1

2. When running models other than ACT, ensure you install the required additional dependencies for those models.


In [ ]:
# For smolVLA
!cd lerobot && pip install -e ".[smolvla]"
# For Pi
!cd lerobot && pip install -e ".[pi]"

3. If you want to resume the training from last checkpoint, run the command below:


In [41]:
!lerobot-train \
  --resume=true \
  --config_path=outputs/train/hackathon_amd_mission2_black_sort/checkpoints/last/pretrained_model/train_config.json

usage: lerobot-train [-h] [--config_path str] [--dataset str]
                     [--dataset.repo_id str] [--dataset.root str]
                     [--dataset.episodes str] [--image_transforms str]
                     [--dataset.image_transforms.enable str]
                     [--dataset.image_transforms.max_num_transforms str]
                     [--dataset.image_transforms.random_order str]
                     [--dataset.image_transforms.tfs str]
                     [--dataset.revision str]
                     [--dataset.use_imagenet_stats str]
                     [--dataset.video_backend str] [--dataset.streaming str]
                     [--env str]
                     [--env.type {aloha,pusht,gym_manipulator,libero,metaworld}]
                     [--env.visualization_width str]
                     [--env.visualization_height str] [--robot str]
                     [--env.robot.type str] [--teleop str]
                     [--env.teleop.type str] [--processor str]
      

4. If you want to upload your dataset using `huggingface-cli upload <repo name> <path to the dataset> --repo-type=dataset`, be sure to set a codebase tag like below:


In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import login

login(token="your_huggingface_token")
hub_api = HfApi()
hub_api.create_tag(<HF_REPO_NAME>, tag="v3.0", revision="main", repo_type="dataset")